In [1]:
using LinearAlgebra
using TrajectoryOptimization
using Plots
plotly()

┌ Info: Precompiling TrajectoryOptimization [c79d492b-0548-5874-b488-5a62c1d9d0ca]
└ @ Base loading.jl:1273
┌ Warning: Package TrajectoryOptimization does not have DifferentialEquations in its dependencies:
│ - If you have TrajectoryOptimization checked out for development and have
│   added DifferentialEquations as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with TrajectoryOptimization
└ Loading DifferentialEquations into TrajectoryOptimization from project dependency, future warnings for TrajectoryOptimization are suppressed.


Plots.PlotlyBackend()

In [2]:
#Pauli spin matrices
i2 = [0 -1; 1 0] #2x2 unit imaginary matrix
Sx = [zeros(2,2) I; I zeros(2,2)]
Sy = [zeros(2,2) -i2; i2 zeros(2,2)]
Sz = [I zeros(2,2); zeros(2,2) -I]
iSx = [zeros(2,2) i2; i2 zeros(2,2)]
iSy = [zeros(2,2) I; -I zeros(2,2)]
iSz = [i2 zeros(2,2); zeros(2,2) -i2];

In [20]:
function qubit_dynamics!(ẋ,x,ü)
      u̇ = x[5]
      u = x[6]
      ẋ .= [(-iSx - u.*iSz)*x[1:4]; ü; u̇; u]
end

qubit_dynamics! (generic function with 1 method)

In [21]:
n = 7 #state dimension
m = 1 #inut dimension
model = Model(qubit_dynamics!,n,m)
model_d = rk4(model);

In [55]:
dt = 0.01 #time step
N = 101 #number of knot points

x0 = [1.0, 0, 0, 0, 0, 0, 0] #initial state
xf = [1/sqrt(2), 0, 1/sqrt(2), 0, 0, 0, 0] #desired final state

u0 = [randn(m) for k = 1:N-1]; #random initial guess for control inputs

#Set up quadratic objective function
Q = Diagonal([1e-3, 1e-3, 1e-3, 1e-3, 1, 1, 1e-3])
R = 1.0*Diagonal(I,m)
Qf = 1.0*Diagonal(I,n)
obj = LQRObjective(Q,R,Qf,xf,N);

In [56]:
#Set up input constriants
goal = goal_constraint(xf) # terminal constraint
constraints = Constraints(N) # define constraints at each time step
constraints[N] += goal;

In [57]:
#Set up and solve traj. opt. problem
prob = Problem(model_d, obj, constraints=constraints, x0=x0, xf=xf, N=N, dt=dt)
initial_controls!(prob, u0) #random guess for initial controls
solver = solve!(prob, ALTROSolverOptions{Float64}());

┌ Info: Augmented Lagrangian solve...
└ @ TrajectoryOptimization /Users/zac/.julia/dev/TrajectoryOptimization/src/solvers/altro/altro_methods.jl:25


In [61]:
X = fill(Float64[], N)
U = zeros(N)
U̇ = zeros(N)
for k = 1:N
      X[k] = prob.X[k][1:4]
      U[k] = prob.X[k][6]
      U̇[k] = prob.X[k][5]
end

In [62]:
plot(X,xlabel="time step",title="State Trajectory",label=["Re(x1)" "Im(x1)" "Re(x2)" "Im(x2)"])

<!DOCTYPE html>
 
 
 Plots.jl

In [66]:
plot(U,xlabel="time step",ylabel="u",title="Control Trajectory",label="u")

<!DOCTYPE html>
 
 
 Plots.jl

In [65]:
plot(U̇,xlabel="time step",ylabel="udot",title="Control Trajectory",label="udot")

<!DOCTYPE html>
 
 
 Plots.jl

In [67]:
plot(prob.U,xlabel="time step",ylabel="uddot",title="Control Trajectory",label="uddot")

<!DOCTYPE html>
 
 
 Plots.jl